In [1]:
# pip install cmake MulticoreTSNE sentence-transformers

In [2]:
# Importing the relevant libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import re
from functools import reduce
  
import bokeh
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_notebook, reset_output
from bokeh.palettes import d3
import bokeh.models as bmo
from bokeh.io import save, output_file
import matplotlib as mpl
from MulticoreTSNE import MulticoreTSNE as TSNE
# init_notebook_mode(connected = True)
# color = sns.color_palette("Set2")
import warnings
warnings.filterwarnings("ignore")

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

import pickle as pkl
import collections, numpy

In [3]:
with open('../../ivr_backup_data/account_embeds.pkl', 'rb') as embed_file:
    embed = pkl.load(embed_file)

In [4]:
embed.shape

(134283, 600)

In [5]:
with open('../../ivr_backup_data/account_keys.pkl', 'rb') as key_file:
    account_keys = pkl.load(key_file)

In [57]:
account_keys.shape

(134283,)

In [7]:
aws_result_df = pd.read_csv('../../ivr_backup_data/active_watch_result.csv')

In [8]:
labels = aws_result_df['ACTIVE_WATCH_RESULT'].values

In [9]:
# labels.shape

In [58]:
reduced_data_set = embed[501:1501, :]    # skip first row which is for "#na#"

In [59]:
reduced_data_set.shape

(1000, 600)

In [60]:
reduced_ids = account_keys[501:1501]

In [61]:
reduced_ids.shape

(1000,)

In [62]:
y = labels[500:1500]

In [63]:
y.shape

(1000,)

In [16]:
# collections.Counter(y)

In [64]:
# need convert y back to colors
def convert_label(label):
    if label == 3:
        return 'RED'
    if label == 2:
        return 'YELLOW'
    if label == 1:
        return 'GREEN'
    raise ValueException       # BLACK

In [65]:
aw_color = []
for label in y:
    aw_color.append(convert_label(label))

In [66]:
len(aw_color)

1000

In [67]:
# aw_color

In [68]:
collections.Counter(aw_color)

Counter({'YELLOW': 85, 'GREEN': 806, 'RED': 109})

In [69]:
acct_id_array = reduced_ids    # skip first row which is for "#na#"

In [70]:
acct_id_array.shape

(1000,)

In [71]:
final_data = reduced_data_set

In [72]:
from bokeh.transform import factor_cmap, factor_mark


In [25]:
# color = '''black
#  cadetblue
#  chartreuse
#  chocolate
#  coral
#  cornflowerblue
#  cornsilk
#  crimson
#  cyan
#  darkblue
#  darkcyan
#  darkgoldenrod
#  darkgray
#  darkgreen
#  darkgrey
#  darkkhaki
#  darkmagenta
#  darkolivegreen
#  darkorange
#  darkorchid
#  darkred
#  darksalmon
#  darkseagreen
#  darkslateblue
#  darkslategray
#  darkslategrey
#  darkturquoise
#  darkviolet
#  deeppink
#  deepskyblue
#  dimgray
#  dimgrey
#  dodgerblue
#  firebrick
#  floralwhite
#  forestgreen
#  fuchsia
#  gainsboro
#  ghostwhite
#  gold
#  goldenrod
#  gray
#  green'''.split()

color = '''green
 yellow
 red
 '''.split()

colormap = np.array(color)

LABELS = ['GREEN', 'YELLOW', 'RED']
MARKERS = ['circle','circle', 'circle']

In [73]:
LABELS

['GREEN', 'YELLOW', 'RED']

In [ ]:
aw_color

In [74]:
colormap

array(['green', 'yellow', 'red'], dtype='<U6')

In [75]:
tsne_model = TSNE(n_jobs=4,
                  perplexity=80,
                  early_exaggeration=4, # Trying out exaggeration trick
                  n_components=2,
                  verbose=1,
                  random_state=2018,
                  n_iter=1000)

tsne_tfidf = tsne_model.fit_transform(np.array(reduced_data_set))

tsne_tfidf_df = pd.DataFrame(data=tsne_tfidf, columns=["x", "y"])

tsne_tfidf_df["account_number"] = reduced_ids
tsne_tfidf_df['cluster'] = aw_color

In [76]:
source = ColumnDataSource(data = dict(x = tsne_tfidf_df["x"], 
                                      y = tsne_tfidf_df["y"],
                                      # color = colormap[clean_data['topics_code'].values.tolist()],
                                      account_number = tsne_tfidf_df["account_number"],
                                      target = tsne_tfidf_df["cluster"]
                                      ))



In [77]:
output_notebook()
plot_tfidf = bp.figure(plot_width = 800, plot_height = 700, 
                       title = "T-SNE applied to User Account Entity Embedding",
                       tools = "pan, wheel_zoom, box_zoom, reset, hover, previewsave",
                       x_axis_type = None, y_axis_type = None, min_border = 1)

plot_tfidf.scatter(x = "x", 
                   y = "y", 
                   legend = "target",
                   source = source,
                   alpha = 0.7, 
                   marker=factor_mark('target', MARKERS, LABELS),
                   color=factor_cmap('target', palette=['green', 'yellow', 'red'], factors=LABELS))


hover = plot_tfidf.select(dict(type = HoverTool))
hover.tooltips = {
                  "account_number": "@account_number",
                    "target": "@target"
                  }

show(plot_tfidf)

Loading BokehJS ...

In [35]:
with open('../../ivr_backup_data/model1_embeds.pkl', 'rb') as model1_embed_file:
    model1_embed = pkl.load(model1_embed_file)

In [36]:
with open('../../ivr_backup_data/model1_keys.pkl', 'rb') as model1_key_file:
    model1_keys = pkl.load(model1_key_file)

In [37]:
len(model1_embed), len(model1_keys)

(64, 64)

In [38]:
tsne_model = TSNE(n_jobs=4,
                  perplexity=80,
                  early_exaggeration=4, # Trying out exaggeration trick
                  n_components=2,
                  verbose=1,
                  random_state=2018,
                  n_iter=1000)

tsne_tfidf = tsne_model.fit_transform(np.array(model1_embed))

tsne_tfidf_df = pd.DataFrame(data=tsne_tfidf, columns=["x", "y"])

tsne_tfidf_df["model_number"] = model1_keys
tsne_tfidf_df['cluster'] = 'blue'

In [43]:
source = ColumnDataSource(data = dict(x = tsne_tfidf_df["x"], 
                                      y = tsne_tfidf_df["y"],
                                      # color = colormap[clean_data['topics_code'].values.tolist()],
                                      model_number = tsne_tfidf_df["model_number"],
                                      target = tsne_tfidf_df["cluster"]
                                      ))


In [44]:
output_notebook()
plot_tfidf = bp.figure(plot_width = 800, plot_height = 700, 
                       title = "T-SNE applied to Model1 Entity Embedding",
                       tools = "pan, wheel_zoom, box_zoom, reset, hover, previewsave",
                       x_axis_type = None, y_axis_type = None, min_border = 1)

plot_tfidf.scatter(x = "x", 
                   y = "y", 
                   legend = "target",
                   source = source,
                   alpha = 0.7, 
                   marker=factor_mark('target', ['circle'], ['blue']),
                   color=factor_cmap('target', palette=['blue'], factors=LABELS))


hover = plot_tfidf.select(dict(type = HoverTool))
hover.tooltips = {
                  "model_number": "@model_number",
                    "target": "@target"
                  }

show(plot_tfidf)

Loading BokehJS ...

In [45]:
with open('../../ivr_backup_data/sku1_embeds.pkl', 'rb') as sku1_embed_file:
    sku1_embed = pkl.load(sku1_embed_file)
with open('../../ivr_backup_data/sku1_keys.pkl', 'rb') as sku1_key_file:
    sku1_keys = pkl.load(sku1_key_file)
len(sku1_embed), len(sku1_keys)

(502, 502)

In [49]:
tsne_sku1_model = TSNE(n_jobs=4,
                  perplexity=80,
                  early_exaggeration=4, # Trying out exaggeration trick
                  n_components=2,
                  verbose=1,
                  random_state=2018,
                  n_iter=1000)

tsne_sku1_tfidf = tsne_sku1_model.fit_transform(np.array(sku1_embed))

tsne_sku1_tfidf_df = pd.DataFrame(data=tsne_sku1_tfidf, columns=["x", "y"])

tsne_sku1_tfidf_df["sku_number"] = sku1_keys
tsne_sku1_tfidf_df['cluster'] = 'blue'

In [55]:
sku_source = ColumnDataSource(data = dict(x = tsne_sku1_tfidf_df["x"], 
                                      y = tsne_sku1_tfidf_df["y"],
                                      # color = colormap[clean_data['topics_code'].values.tolist()],
                                      sku_number = tsne_sku1_tfidf_df["sku_number"],
                                      target = tsne_sku1_tfidf_df["cluster"]
                                      ))


In [56]:
output_notebook()
plot_tfidf = bp.figure(plot_width = 800, plot_height = 700, 
                       title = "T-SNE applied to SKU1 Entity Embedding",
                       tools = "pan, wheel_zoom, box_zoom, reset, hover, previewsave",
                       x_axis_type = None, y_axis_type = None, min_border = 1)

plot_tfidf.scatter(x = "x", 
                   y = "y", 
                   legend = "target",
                   source = sku_source,
                   alpha = 0.7, 
                   marker=factor_mark('target', ['circle'], ['blue']),
                   color=factor_cmap('target', palette=['blue'], factors=['blue']))


hover = plot_tfidf.select(dict(type = HoverTool))
hover.tooltips = {
                  "sku_number": "@sku_number",
                    "target": "@target"
                  }

show(plot_tfidf)

Loading BokehJS ...